# Este codigo prepara los datos para usarlos con pytorch (generando un archivo CSV)

In [154]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [155]:
# consts
cats = ["MEL","NV","BCC","AK","BKL","DF","VASC","SCC","UNK"]

train_percent = .7
val_percent = .2
test_percent = .1

## Sacamos los datos del CSV

In [156]:
csv = pd.read_csv("./dataset/ISIC_2019_Training_GroundTruth.csv")

print("> Cantidad de elementos", csv.count(axis=1).size )
print("> Head de los datos del csv")
print(csv.head())

> Cantidad de elementos 25331
> Head de los datos del csv
          image  MEL   NV  BCC   AK  BKL   DF  VASC  SCC  UNK
0  ISIC_0000000  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
1  ISIC_0000001  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
2  ISIC_0000002  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
3  ISIC_0000003  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
4  ISIC_0000004  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0


## Sacamos las categorias (esto no es necesario)

In [157]:
header = list(csv.columns)
header.remove("image")

print("> Categorias")
print(header)

> Categorias
['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']


## Creamos un Dataframe en el formato que requiere pytorch (imagen<string>, categoria<int>)

In [158]:
data = pd.DataFrame({"img":[], "cat": []}, dtype=int)

for entry in csv.values:
    data = data.append({"img":entry[0] , "cat":np.where(entry==1.0)[0][0]-1}, ignore_index=True)


print("> Cantidad de filas", csv.count(axis=1).size)
print("> Head de los datos en el formato que usa pytorch")
print(data.head())

> Cantidad de filas 25331
> Head de los datos en el formato que usa pytorch
            img  cat
0  ISIC_0000000    1
1  ISIC_0000001    1
2  ISIC_0000002    0
3  ISIC_0000003    1
4  ISIC_0000004    0


## Contamos cuantas imagenes hay de cada categoria

In [159]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

> Cantidad de imagenes de cada tipo:
[ 4522 12875  3323   867  2624   239   253   628     0]


## Averiguamos cual es el tamaño del grupo de imagenes mas pequeño

In [160]:
# counter = list(filter(lambda elm: elm > 0, counter))
# min_cat_size = min(counter)

min_cat_size = data.cat.value_counts().min()

print("> Cantidad mínima de imagenes de un tipo:")
print(min_cat_size)

> Cantidad mínima de imagenes de un tipo:
239


## Hacemos que todos los grupos tengan el mismo tamaño, eligiendolos de forma aleatoria

In [161]:
data = pd.DataFrame(data.groupby("cat").apply(lambda cat: cat.sample(min_cat_size)).reset_index(drop=True))

print("> Cantidad de elemntos:", data.count(axis=1).size )
print("> Datos equilibrados")
print(data.head())

> Cantidad de elemntos: 1912
> Datos equilibrados
                        img  cat
0  ISIC_0014372_downsampled    0
1              ISIC_0058177    0
2              ISIC_0073237    0
3  ISIC_0014072_downsampled    0
4              ISIC_0055405    0


## Barajamos todas las filas del DataFrame

In [162]:
data = data.sample(frac=1)

print("> cantidad de elementos", data.count(axis=1).size)
print("> datos barajados")
print(data.head())

> cantidad de elementos 1912
> datos barajados
               img  cat
842   ISIC_0028370    3
1777  ISIC_0064287    7
1881  ISIC_0058934    7
1260  ISIC_0031799    5
195   ISIC_0056389    0


## Comprobamos cuantas imagenes hay de cada categoria

In [163]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

> Cantidad de imagenes de cada tipo:
[239 239 239 239 239 239 239 239   0]


## Guardamos el resultado en un archivo CSV
> Esto para mas eficiencia a la hora de usarlo, mantener siempre el mismo dataset y evitar la sobrerrepresentación de las categorias con mas elementos.

In [164]:
data.to_csv("./dataset/balanced_data.csv", index=False)

## Ahora vamos a separar los datos en 3 grupos (train, val, test)

In [165]:
train_data, tmp = train_test_split(data, train_size=train_percent, stratify=data['cat'], shuffle=True)
val_data, test_data = train_test_split(tmp, test_size=test_percent/(test_percent+val_percent), stratify=tmp['cat'], shuffle=True)


print("> train", train_data.count(axis=1).size)
print(test_data.head())
print()

print("> val", val_data.count(axis=1).size)
print(val_data.head())
print()

print("> test", test_data.count(axis=1).size)
print(test_data.head())
print()

> train 1338
                           img  cat
465               ISIC_0031744    1
1288              ISIC_0034169    5
1615              ISIC_0030722    6
782               ISIC_0066391    3
444   ISIC_0015161_downsampled    1

> val 382
              img  cat
471  ISIC_0066017    1
382  ISIC_0064461    1
660  ISIC_0062225    2
704  ISIC_0034095    2
433  ISIC_0054256    1

> test 192
                           img  cat
465               ISIC_0031744    1
1288              ISIC_0034169    5
1615              ISIC_0030722    6
782               ISIC_0066391    3
444   ISIC_0015161_downsampled    1



## Comprobamos cuantas imagenes hay de cada categoria para cada grupo

In [166]:
counter = np.zeros(len(cats), dtype=int)
for elm in train_data.values:
    counter[int(elm[1])]+=1

print("> train", train_data.count(axis=1).size, train_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/train_data.count(axis=1).size})")
print()

counter = np.zeros(len(cats), dtype=int)
for elm in val_data.values:
    counter[int(elm[1])]+=1

print("> val", val_data.count(axis=1).size, val_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/val_data.count(axis=1).size})")
print()

counter = np.zeros(len(cats), dtype=int)
for elm in test_data.values:
    counter[int(elm[1])]+=1

print("> test", test_data.count(axis=1).size, test_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/test_data.count(axis=1).size})")
print()

> train 1338 0.6997907949790795
[168 167 168 167 167 167 167 167   0] (0.12556053811659193)

> val 382 0.1997907949790795
[47 48 47 48 48 48 48 48  0] (0.12303664921465969)

> test 192 0.100418410041841
[24 24 24 24 24 24 24 24  0] (0.125)



## Barajamos los datos

In [152]:
train_data = train_data.sample(frac=1)
val_data = val_data.sample(frac=1)
test_data = test_data.sample(frac=1)

print("> train", train_data.count(axis=1).size)
print(train_data.head(), end="\n\n")

print("> val", val_data.count(axis=1).size)
print(val_data.head(), end="\n\n")

print("> test", test_data.count(axis=1).size)
print(test_data.head(), end="\n\n")

> train 1338
                           img  cat
1705              ISIC_0069086    7
1265              ISIC_0028651    5
1312              ISIC_0025668    5
1172  ISIC_0012364_downsampled    4
1302              ISIC_0030579    5

> val 382
               img  cat
1886  ISIC_0064016    7
1000  ISIC_0057348    4
1176  ISIC_0032694    4
70    ISIC_0065421    0
1283  ISIC_0026313    5

> test 192
                           img  cat
233   ISIC_0012395_downsampled    0
1161              ISIC_0059118    4
1506              ISIC_0069242    6
186               ISIC_0068011    0
260               ISIC_0028922    1



## Guardamos cada uno de los dataframes en un csv

In [153]:
train_data.to_csv("dataset/train_data.csv", index=False)
val_data.to_csv("dataset/val_data.csv", index=False)
test_data.to_csv("dataset/test_data.csv", index=False)